In [1]:
!export TMPDIR=/ML-A800/home/xiangyue/yfli/tmp
!export HF_HOME=/ML-A800/home/xiangyue/yfli/.cache/huggingface
!export CUDA_VISIBLE_DEVICES=4,5,6,7

In [5]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

model_name = "/ML-A800/models/Mixtral-8x7B-Instruct-v0.1"
# model_name = "/ML-A800/models/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                            # attn_implementation="flash_attention_2",   #You can use flash attention on your local GPU with specific libraries
                                             )

Loading checkpoint shards: 100%|██████████| 19/19 [00:27<00:00,  1.42s/it]


In [6]:
tokenizer.pad_token = "!" #Not EOS, will explain another time.
CUTOFF_LEN = 256  #Our dataset has shot text
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1

In [7]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[ "w1", "w2", "w3"],  #Only Training the "expert" layers
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [8]:
def print_trainable_parameters(m):
    trainable_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in m.parameters())
    print(f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}")

print_trainable_parameters(model)

trainable params: 113246208 || all params: 23595847680 || trainable%: 0.4799412571898752


In [ ]:
!export http_proxy=100.66.28.72:3128
!export https_proxy=100.66.28.72:3128
!export HTTP_PROXY=100.66.28.72:3128
!export HTTPS_PROXY=100.66.28.72:3128

In [9]:
dataset = load_dataset("/ML-A800/home/xiangyue/yfli/hf_datasets/modern-to-shakesperean-translation") #Found a good small dataset for a quick test run!
print("dataset", dataset)
train_data = dataset["train"] # Not using evaluation data

dataset DatasetDict({
    train: Dataset({
        features: ['modern', 'shakespearean'],
        num_rows: 274
    })
})


In [10]:
def generate_prompt(user_query,  sep="\n\n### "):  #The prompt format is taken from the official Mixtral huggingface page
    sys_msg= "Translate the given text to Shakespearean style."
    p =  " [INST]" + sys_msg +"\n"+ user_query["modern"] + "[/INST]" +  user_query["shakespearean"] + ""
    return p

In [11]:
def tokenize(prompt):
    return tokenizer(
        prompt + tokenizer.eos_token,
        truncation=True,
        max_length=CUTOFF_LEN ,
        padding="max_length"
    )

In [12]:
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=["modern" , "shakespearean"])

Map: 100%|██████████| 274/274 [00:00<00:00, 3223.56 examples/s]


In [ ]:
tokenize(generate_prompt(train_data[0]))

In [ ]:
tokenizer.decode([1,2])

In [ ]:
tokenizer.convert_ids_to_tokens(tokenize(generate_prompt(train_data[0]))["input_ids"])

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=6,
        learning_rate=1e-4,
        logging_steps=2,
        optim="adamw_torch",
        save_strategy="epoch",
        output_dir="mixtral-moe-lora-instruct-shapeskeare"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

Detected kernel version 5.4.210, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

In [ ]:
trainer.train()

In [ ]:
from transformers import MixtralModel, MixtralConfig

configuration = MixtralConfig()

In [ ]:
configuration

In [ ]:
tokenizer.bos_token